In [14]:
import scanpy as sc
import scvi
import numpy as np
import pandas as pd
import seaborn as sns
import os

In [39]:
def pp(sample_path):
    # Load the sample data from .h5 file
    adata = sc.read_10x_h5(sample_path)
    adata.var_names_make_unique()

    # Continue with your analysis steps as before
    sc.pp.filter_genes(adata, min_cells=10)
    sc.pp.highly_variable_genes(adata, n_top_genes=2000, subset=True, flavor='seurat_v3')
    scvi.model.SCVI.setup_anndata(adata)
    vae = scvi.model.SCVI(adata)
    vae.train()
    solo = scvi.external.SOLO.from_scvi_model(vae)
    solo.train()
    df = solo.predict()
    df['prediction'] = solo.predict(soft=False)
    df.index = df.index.map(lambda x: x[:-2])
    df['dif'] = df.doublet - df.singlet
    doublets = df[(df.prediction == 'doublet') & (df.dif > 1)]

    adata.obs['Sample'] = os.path.basename(sample_path)  # Extract the filename as the sample name

    adata.obs['doublet'] = adata.obs.index.isin(doublets.index)
    adata = adata[~adata.obs.doublet]

    sc.pp.filter_cells(adata, min_genes=200)
    adata.var['mt'] = adata.var_names.str.startswith('Mt-')
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
    
    # You can return or do further processing with 'adata' for each sample here
    # For example, you can save it to a list or dictionary for later use.

    return adata


In [42]:
sample_names = [
    'cE14F1.h5', 'cGFPF1.h5', 'eERMF1.h5', 'eSTRF1.h5',
    'cE14F2.h5', 'cGFPF2.h5', 'eERMF2.h5', 'eSTRF2.h5',
    'cE14F3.h5', 'cGFPF3.h5', 'eERMF3.h5', 'eSTRF3.h5',
    'cE14M1.h5', 'cGFPM1.h5', 'eERMM1.h5', 'eSTRM1.h5',
    'cE14M2.h5', 'cGFPM2.h5', 'eERMM2.h5', 'eSTRM2.h5',
    'cE14M3.h5', 'cGFPM3.h5', 'eERMM3.h5', 'eSTRM3.h5',
    'lERMM3.h5', 'lSTRM3.h5', 'lERMF1.h5', 'lSTRF1.h5',
    'lERMF2.h5', 'lSTRF2.h5', 'lERMF3.h5', 'lSTRF3.h5',
    'lERMM1.h5', 'lSTRM1.h5', 'lERMM2.h5', 'lSTRM2.h5'
]

# Base directory where the files are located
base_dir = '/home/zia/hippo_h5_files'

# Generate the list of full file paths
sample_paths = [os.path.join(base_dir, sample_name) for sample_name in sample_names]

# Create an empty list to store the processed data for each sample
processed_data_list = []

# Loop through the sample paths and process each sample
for sample_path in sample_paths:
    processed_data = pp(sample_path)
    processed_data_list.append(processed_data)

# Now, processed_data_list contains the processed data for all samples.
# You can access and analyze each processed dataset as needed.


/home/zia/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/zia/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/zia/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/zia/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/zia/.local/lib/python3.10/site-packages/anndata/_core/an

Epoch 400/400: 100%|█| 400/400 [11:42<00:00,  1.73s/it, v_num=1, train_loss_step

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|█| 400/400 [11:42<00:00,  1.76s/it, v_num=1, train_loss_step
INFO     Creating doublets, preparing SOLO model.                                                                  


/home/zia/.local/lib/python3.10/site-packages/anndata/_core/merge.py:217: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(dtype):
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


...

/home/zia/.local/lib/python3.10/site-packages/anndata/_core/merge.py:217: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(dtype):
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Epoch 219/400:  55%|▌| 219/400 [01:12<00:59,  3.04it/s, v_num=1, train_loss_step
Monitored metric validation_loss did not improve in the last 30 records. Best score: 0.308. Signaling Trainer to stop.


/home/zia/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/zia/.local/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:139: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number
/home/zia/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/zia/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/zia/.local/lib/python3.10/site-packages/anndata/_c

Epoch 1/400:   0%|                                      | 0/400 [00:00<?, ?it/s]

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 128])

In [58]:
processed_data_list

[AnnData object with n_obs × n_vars = 4536 × 2000
     obs: '_scvi_batch', '_scvi_labels', 'Sample', 'doublet', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
     uns: 'hvg', '_scvi_uuid', '_scvi_manager_uuid',
 AnnData object with n_obs × n_vars = 4006 × 2000
     obs: '_scvi_batch', '_scvi_labels', 'Sample', 'doublet', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
     uns: 'hvg', '_scvi_uuid', '_scvi_manager_uuid',
 AnnData object with n_obs × n_vars = 4201 × 2000
    